# iOS Message Extractor and Formatter

In [ ]:
#http://bugcharmer.blogspot.com/2015/02/exporting-text-messages-from-iphone.html
#https://sweet-as-tandy.com/2015/06/26/how-to-retrieve-and-analyze-your-ios-messages-with-python-pandas-and-nltk/
#https://osxdaily.com/2010/07/08/read-iphone-sms-backup/
#https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html
#https://stackoverflow.com/questions/39541908/convert-cocoa-timestamp-in-python
#https://codepen.io/swards/pen/gxQmbj
#https://apple.stackexchange.com/questions/77432/location-of-message-attachments-in-ios-6-backup
#https://www.richinfante.com/2017/3/16/reverse-engineering-the-ios-backup#manifestdb


import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import numpy as np
import os
import hashlib
from shutil import copy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None

In [ ]:
#
# Set the variables
#

correspondence_name = 'John Doe'
path_to_ios_backup = '/Users/johngrinalds/Library/Application Support/MobileSync/Backup/6ef7a452078bbfb82a077376a7c9c62056d398d2/'
path_to_address_book = '/Users/johngrinalds/Library/Application Support/MobileSync/Backup/6ef7a452078bbfb82a077376a7c9c62056d398d2/31/31bb7ba8914766d4ba40d6dfb6113c8b614be442'
path_to_messages = '/Users/johngrinalds/Library/Application Support/MobileSync/Backup/6ef7a452078bbfb82a077376a7c9c62056d398d2/3d/3d0d7e5fb2ce288813306e4d4636395e047a3d28'
path_to_voicemails = '/Users/johngrinalds/Library/Application Support/MobileSync/Backup/6ef7a452078bbfb82a077376a7c9c62056d398d2/99/992df473bbb9e132f4b3b6e4d33f72171e97bc7a'
path_to_manifest = '/Users/johngrinalds/Library/Application Support/MobileSync/Backup/6ef7a452078bbfb82a077376a7c9c62056d398d2/Manifest.db'
attachments_destination = "/Users/johngrinalds/Documents/GitHub/ios-message-extractor/attachments"
voicemail_destination = "/Users/johngrinalds/Documents/GitHub/ios-message-extractor/voicemails"

# Gather and clean tables

In [ ]:
# This function is used to convert the iOS timestamp into a readable format
def ts_convert(t):
    unix = datetime(1970, 1, 1)  # UTC
    cocoa = datetime(2001, 1, 1)  # UTC

    delta = cocoa - unix  # timedelta instance

    timestamp = datetime.fromtimestamp(int(t/1000000000)) + delta

    return timestamp.strftime('%A, %B %d, %Y %I:%M %p')

In [ ]:
# Get the names and numbers from the address book
con = sqlite3.connect(path_to_address_book)
df_ab = pd.read_sql_query("SELECT * FROM ABPersonFullTextSearch_content", con)
con.close()

#This function converts Nones to blanks
def xstr(s):
    return '' if s is None else str(s)

df_ab['c0First'] = df_ab['c0First'].apply(lambda x: xstr(x))
df_ab['c1Last'] = df_ab['c1Last'].apply(lambda x: xstr(x))

df_ab = df_ab[['c0First','c1Last','c16Phone']]

df_ab['name'] = df_ab.c0First + ' ' + df_ab.c1Last
df_ab['phone'] = df_ab['c16Phone'].apply(lambda x: str(x)[-19:-9]) #using the 10 digit number as an identifier
df_ab = df_ab[['name','phone']]
df_ab.drop_duplicates(inplace = True)
df_ab.drop_duplicates(inplace = True, subset = ['phone']) #there are some people who are shown to have the same number...

df_ab['name'].replace('', np.nan, inplace=True)
df_ab['name'].replace(' ', np.nan, inplace=True) #some names have large spaces in them
df_ab['phone'].replace('', np.nan, inplace=True)
df_ab = df_ab.dropna()

#df_ab

In [ ]:
# Get the handles of each phone number
con = sqlite3.connect(path_to_messages)
df_handle = pd.read_sql_query("SELECT * FROM handle", con)
con.close()

df_handle['id'] = df_handle['id'].apply(lambda x: str(x)[-10:])
df_handle = df_handle[['ROWID','id']]
df_handle = df_handle.rename(columns = {'id':'phone', 'ROWID':'handle_id'})
df_handle.dropna(inplace = True)
#df_handle

In [ ]:
# Get the message content
con = sqlite3.connect(path_to_messages)
df_mess = pd.read_sql_query("SELECT * FROM message", con)
con.close()

df_mess = df_mess[['ROWID','handle_id','text','is_from_me','date','cache_roomnames']]
df_mess['date'] = df_mess['date'].apply(lambda d: ts_convert(d))
df_mess['display_date'] = df_mess['date']
df_mess['date'] = pd.to_datetime(df_mess['date'])
df_mess = df_mess[~df_mess['text'].str.contains('Laughed at “|Liked “|Loved “|Emphasized “|Emphasized an image|Loved an image|Liked an image', regex=True).astype('bool')] # This line removes the reactions from the text; note the peculiar double quotes...
df_mess = df_mess[df_mess['cache_roomnames'].isna()] # This line filters out group conversations
df_mess.reset_index(inplace = True, drop = True)
#df_mess

In [ ]:
# Attachments Join Table
con = sqlite3.connect(path_to_messages)
df_att_key = pd.read_sql_query("SELECT * FROM message_attachment_join", con)
con.close()

#df_att_key

In [ ]:
# Attachments

#Connect to the attachment table
con = sqlite3.connect(path_to_messages)
df_att = pd.read_sql_query("SELECT * FROM attachment", con)
con.close()

df_att = df_att[['ROWID','filename','mime_type','transfer_name']]
df_att = df_att[df_att['mime_type'].str.contains('jpeg|png|heic|gif', regex=True).astype('bool')] # filter to only the image attachements
df_att['ext'] = df_att['mime_type'].apply(lambda x: str(x).split('image/', 1)[1]) #get the file extension
df_att['filename_new'] = df_att['filename'].apply(lambda x: 'MediaDomain-Library/' + str(x)[10:]) # this is the filename needed for the SHA1 hash
df_att['sha1'] = df_att['filename_new'].apply(lambda x: hashlib.sha1(str.encode(x)).hexdigest())

# This function will return the path of the attachment files referenced in the messages
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

df_att['filepath'] = df_att['sha1'].apply(lambda x: find(x, path_to_ios_backup))
df_att = df_att[~df_att['filepath'].isna()] # Remove the entries for the files that couldn't be found
df_att = df_att[['ROWID','ext','sha1','filepath']]
#df_att

# Join Tables

In [ ]:
# Merge the attachment information with the attachment key
df_att_merge = pd.merge(df_att, df_att_key, how = 'left', left_on = 'ROWID', right_on = 'attachment_id')
df_att_merge.drop(columns = ['ROWID','attachment_id'], inplace = True)
df_att_merge.dropna(inplace = True)
df_att_merge['message_id'] = df_att_merge['message_id'].astype('int')
df_att_merge

In [ ]:
# Merge the names with the handles via the phone number
df_merge = None
df_merge = pd.merge(df_ab, df_handle, how = 'left', on = 'phone')
df_merge.dropna(inplace = True)
df_merge['handle_id'] = df_merge['handle_id'].astype(int)
df_merge

In [ ]:
# Merge the messages with the names via the handle
df_final = pd.merge(df_mess, df_merge, how = 'left' , on = 'handle_id')
df_final.drop(columns = ['handle_id','phone'], inplace = True)
df_final

In [ ]:
# Merge the messages with the attachements
df_final = pd.merge(df_final, df_att_merge, how = 'left' , left_on = 'ROWID', right_on = 'message_id')
df_final.drop(columns = ['message_id'], inplace = True)
df_final

# Parse data into HTML

In [ ]:
# This function will copy the image attachments from the backup folder,
# convert any .heic files to .jpegs, and then delete the .heic files.

def copy_rename_attachments(filepath, dest, sha1, ext):
    copy(filepath, dest)
    os.rename("attachments/" + sha1,"attachments/" + sha1 + "." + ext)
    os.system('magick mogrify -monitor -format jpeg attachments/*.heic')
    os.system('find attachments/ -name "*.heic" -delete')

In [ ]:
def generate_text_history(name, df, swap_sides = False):
    
    # create the directory to store the attachment images
    if not os.path.exists('./attachments'):
        os.mkdir('./attachments')
    
    # The "swap sides" parameters will switch who is rendered as the "sender" and "receiver" in the HTML
    if swap_sides == True:
        from_me_flag = 0
    else:
        from_me_flag = 1
    
    df = df[df['name'] == name] # filter to the name of the person you corresponded with
    df = df.reset_index()
    
    content = ''

    for i in range(len(df['name'])):
               
        
        #insert dates
        
        if i == 0:
            content = content + '''
                <div class="date">
                  {}
                 </div>

                '''.format(df['display_date'][i])
        else:
            if df['date'][i] > df['date'][i-1] + timedelta(minutes = 10):
                content = content + '''
                <div class="date">
                  {}
                 </div>

                '''.format(df['display_date'][i])


        if df['is_from_me'][i] == from_me_flag:
            content = content + '''
              <div class="mine messages">
                <div class="message">
            '''
            
            if pd.notnull(df['filepath'][i]):
                copy_rename_attachments(df['filepath'][i], attachments_destination, df['sha1'][i], df['ext'][i])
                df['ext'].loc[i] = df['ext'].loc[i].replace('heic','jpeg')
                image_src = "attachments/" + df['sha1'][i] + "." + df['ext'][i]
                content = content + '<img src ="{}" height="6">'.format(image_src)
            
            content = content + '''
              {}
            </div>
          </div>
          '''.format(df['text'].iloc[i])
            
        else:
            content = content + '''
          <div class="yours messages">
            <div class="message">
            '''
            
            if pd.notnull(df['filepath'][i]):
                copy_rename_attachments(df['filepath'][i], attachments_destination, df['sha1'][i], df['ext'][i])
                df['ext'].loc[i] = df['ext'].loc[i].replace('heic','jpeg')
                image_src = "attachments/" + df['sha1'][i] + "." + df['ext'][i]
                content = content + '<img src ="{}" height="6">'.format(image_src)
            
            content = content + '''
              {}
            </div>
          </div>
          '''.format(df['text'].iloc[i])
            
    return content

In [ ]:
with open('css.txt', 'r') as file:
    css = file.read().replace('\n', '')
    
html_output = css + '''
<body>
<h1>{} iOS Message History</h1>
<div class="chat">

  {}  
  
</div>
</body>
'''.format(correspondence_name, generate_text_history(correspondence_name, df_final, swap_sides = False))

#Uncomment this line you want to view the output in Jupyter
#display(HTML(html_output))

In [ ]:
# Output to file
finalHTMLString = ''
for i in range(len(html_output)):
    finalHTMLString += html_output[i].replace('\n','')

Html_file= open("{}_iOS_messages.html".format(correspondence_name.replace(' ','_')),"w")
Html_file.write(finalHTMLString)
Html_file.close()

# Voicemail

In [ ]:
# Manifest

con = sqlite3.connect(path_to_manifest)
df_mf_vm = pd.read_sql_query("SELECT * from FILES where relativePath like 'Library/Voicemail/%.amr'", con)
con.close()

df_mf_vm = df_mf_vm[['fileID','relativePath']]
df_mf_vm['ROWID'] = df_mf_vm['relativePath'].apply(lambda x: os.path.split(x)[1][:-4]).astype('int') # Get the ID of the voicemail file

In [ ]:
# Tables are map, voicemail, and deleted

con = sqlite3.connect(path_to_voicemails)
df_vm = pd.read_sql_query("SELECT *, datetime(date, 'unixepoch','localtime') AS XFORMATTEDDATESTRING from voicemail ORDER BY date ASC", con)
con.close()

df_vm = df_vm[['ROWID','sender','XFORMATTEDDATESTRING']]
df_vm['sender'] = df_vm['sender'].str[2:]
df_vm = pd.merge(df_vm, df_merge[['name','phone']].drop_duplicates(), how = 'left', left_on = 'sender', right_on = 'phone')
df_vm = df_vm[['ROWID','XFORMATTEDDATESTRING','name','phone']]
df_vm[df_vm['ROWID'] ==55]
df_vm

In [ ]:
df_vm_merged = pd.merge(df_vm, df_mf_vm, how = 'left', on = 'ROWID')
df_vm_merged['ROWID'].duplicated(keep = 'first')
df_vm_merged['filepath'] = df_vm_merged['fileID'].apply(lambda x: find(x, path_to_ios_backup))
df_vm_merged['final'] = (df_vm_merged['XFORMATTEDDATESTRING']+ ' ' +df_vm_merged['name']+ ' ' +df_vm_merged['phone']+'.amr').astype('str')
df_vm_merged['final'] = np.where(df_vm_merged['final'] == 'nan',df_vm_merged['XFORMATTEDDATESTRING']+'.amr',df_vm_merged['final'])
df_vm_merged['final'] = df_vm_merged['final'].apply(lambda x: x.replace(' ','_').replace(':','.'))
df_vm_merged

In [ ]:
def copy_voicemail(filepath, dest, sha1, final):
    copy(filepath, dest)
    os.rename("voicemails/" + sha1,"voicemails/" + final)

In [ ]:
# Go through all the voicemails, copy to the folder, and change the filename to the correct name
for i in range(len(df_vm_merged['final'])):
    copy_voicemail(df_vm_merged['filepath'].iloc[i],voicemail_destination ,df_vm_merged['fileID'].iloc[i],df_vm_merged['final'].iloc[i])